In [9]:
import more_itertools as mit
import transformers
import torch
import rich
import rich.panel
import rich.markup
import re
import collections
import numpy as np
import random
import datasets


In [5]:
m = transformers.pipeline(
    "text-generation",
    model="HuggingFaceTB/SmolLM2-1.7B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_new_tokens=512,
)

Device set to use cuda:0


In [18]:
was_good = []
i_numbers = list(range(0, 9999))
random.shuffle(i_numbers)
i_numbers = i_numbers[:100]

j_numbers = list(range(0, 9999))
random.shuffle(j_numbers)
j_numbers = j_numbers[:100]


pairs = []
for i in i_numbers:
    for j in j_numbers:
        pairs.append((i, j))

random.shuffle(pairs)

data = datasets.Dataset.from_dict({"pair": pairs, "questions": [
    f"What is {' '.join(list(str(joined_i)))} + {' '.join(list(str(joined_j)))} = ?" for joined_i, joined_j in pairs
]})


for batch in data.batch(10):
    print(batch)
    
    questions = batch["questions"]
    pairs = batch["pair"]

    outputs = m(
        [[{
            "role": "user", 
            "content": question,
        }] for question in questions], 
        do_sample=False,
    )

    answers = []
    for one_output in outputs:
        assert one_output.keys() == set(["generated_text"]), (
            one_output.keys(), set(["generated_text"])
        ) 
        one_message = one_output["generated_text"][-1]["content"]
        extracted = mit.last(re.findall(r"\d+", one_message), None)

        if extracted is not None:
            answers.append(extracted)
    
    reference = str(i + j)
    selected = str(collections.Counter(answers).most_common(1)[0][0]).strip()
    was_good.append(selected == reference)

    print(f"extracted: {selected}")
    print(f"reference: {reference}")
    print(f"average:   {np.mean(was_good):0.1%}")
    print(f"qty:       {len(was_good)}")
    print("-" * 20)


Batching examples:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'pair': [[5591, 3195], [9650, 7017], [3578, 171], [620, 9453], [5267, 7884], [1767, 7247], [3800, 8110], [6580, 789], [6380, 7756], [1147, 6873]], 'questions': ['What is 5 5 9 1 + 3 1 9 5 = ?', 'What is 9 6 5 0 + 7 0 1 7 = ?', 'What is 3 5 7 8 + 1 7 1 = ?', 'What is 6 2 0 + 9 4 5 3 = ?', 'What is 5 2 6 7 + 7 8 8 4 = ?', 'What is 1 7 6 7 + 7 2 4 7 = ?', 'What is 3 8 0 0 + 8 1 1 0 = ?', 'What is 6 5 8 0 + 7 8 9 = ?', 'What is 6 3 8 0 + 7 7 5 6 = ?', 'What is 1 1 4 7 + 6 8 7 3 = ?']}


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
import rich.markdown

In [37]:
rich.print(rich.markdown.Markdown(
    f"""
**heheheh**: loool
- a
- b
- c
""".strip()
))

heheheh: loool                                                                                                     

 • a                                                                                                               
 • b                                                                                                               
 • c